In [8]:
# importing os module 
import os
#import the pandas library and aliasing as pd
import pandas as pd

In [2]:
def yesno(question):
    """Simple Yes/No Function."""
    prompt = f'{question} ? (y/n): '
    ans = input(prompt).strip().lower()
    if ans not in ['y', 'n']:
        print(f'{ans} is invalid, please try again...')
        return yesno(question)
    if ans == 'y':
        return True
    return False

In [3]:
def GenerateData(fname, path):
    rmsdpp_file = open(path + '/rmsdpp.dat', "x")
    with open(fname, 'r') as f:
        for line in f.readlines():
            if 'RMSDPP' in line:
                rmsdpp_file.write(str(line.split()[-1]) + '\n')
    rmsdpp_file.close()


    rmsdpe_file = open(path + '/rmsdpe.dat', "x")
    with open(fname, 'r') as f:
        for line in f.readlines():
            if 'RMSDPE' in line:
                rmsdpe_file.write(str(line.split()[-1]) + '\n')
    rmsdpe_file.close()


    energy_file = open(path + '/energy.dat', "x")
    with open(fname, 'r') as f:
        for line in f.readlines():
            if ('smt' in fname):
                if ('The Energy for This Iteration is:' in line):
                    energy_file.write(str(line.split()[6]) + '\n')
            else:
                if 'SubSCF Loop, Total Energy = ' in line:
                    energy_file.write(str(line.split()[7]) + '\n')
    energy_file.close()

In [4]:
cwd = os.getcwd()

log_files = []

for file in os.listdir(cwd):
    filename = os.fsdecode(file)
    if filename.endswith(".log"):
        total_name = os.path.join(cwd, filename)
        no_path_name = total_name.split("/")[-1] 
        print(no_path_name)
        log_files.insert(0,str(no_path_name))

hcn_smt_ccpvdz_pb4f1.log
hcn_stw_ccpvtz_pb4f1.log
hcn_stw_ccpvqz_pb4f1.log
hcn_stw_ccpvdz_pb4f1.log
hcn_smt_ccpvtz_pb4f1.log
hcn_smt_ccpvqz_pb4f1.log


In [5]:
log_files

['hcn_smt_ccpvqz_pb4f1.log',
 'hcn_smt_ccpvtz_pb4f1.log',
 'hcn_stw_ccpvdz_pb4f1.log',
 'hcn_stw_ccpvqz_pb4f1.log',
 'hcn_stw_ccpvtz_pb4f1.log',
 'hcn_smt_ccpvdz_pb4f1.log']

In [6]:
saveData = yesno("Do you want to save the data")
print(f'you answer was: {saveData}')

Do you want to save the data ? (y/n): n
you answer was: False


In [7]:
if (saveData):
    for log_file in log_files:
        destination_dir = log_file[:-4]
        all_data_storage_dir = '/Users/aodongliu/LiGroup/gaussian/test_py_data_processing/Data'
        path = os.path.join(all_data_storage_dir, destination_dir)
        os.mkdir(path)
        GenerateData(log_file,path)

In [17]:
name = 'hcn_smt_ccpvqz_pb4f1.log'

name.split('_')

['hcn', 'smt', 'ccpvqz', 'pb4f1.log']

In [14]:
def getInfo(log_file):
        
    Filename = log_file[:-4]
    
    Structure = log_file.split('_')[0]
    
    
    
    


In [ ]:
Data = []
for log_file in log_files:

In [13]:
df = pd.DataFrame(columns=['Filename',
                          'Structure',
                          'EBasis',
                          'PBasis',
                          'NEBasis',
                          'NPBasis',
                          'NIter-SMT',
                          'NIter-STW-E',
                          'NIter-STW-P',
                          ])
print (df)

Empty DataFrame
Columns: [Filename, Structure, EBasis, PBasis, NEBasis, NPBasis, NIter-SMT, NIter-STW-E, NIter-STW-P]
Index: []
